<a href="https://colab.research.google.com/github/firekuo0803/Selflearning_AI/blob/main/AI_painting_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###函式庫下載和匯入

In [ ]:
!pip install opendatasets --upgrade --quiet
!pip install jovian --upgrade --quiet
%matplotlib inline

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import opendatasets as od


##AAAAA

In [ ]:
import os
from google.colab import drive
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as T
import torch
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import torch.nn as nn
from torchvision.utils import save_image
from tqdm.notebook import tqdm
import torch.nn.functional as F

###參數設定

In [ ]:
#專案資料夾名稱
filenames ="anime_dataset"

#資料庫網址
dataset_url = 'https://www.kaggle.com/datasets/scribbless/another-anime-face-dataset'

#圖片參數設定
image_size = 64
batch_size = 128
stats = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)

#資料庫資料夾位置
# DATA_DIR = '/content/gdrive/MyDrive/anime_dataset/B'

#生成圖片儲存位置
sample_dir = 'generated7'
os.makedirs(sample_dir, exist_ok=True)

#學習率
lr = 0.0004

#算圖次數
epochs = 900

latent_size = 128

g_num =64

###連google

In [ ]:
drive.mount("/content/gdrive")
path = "/content/gdrive/MyDrive"
os.chdir(path) #切換到要存取資料夾

#創建資料夾
if not os.path.isdir(filenames):
  os.makedirs(filenames)
path2 = f"/content/gdrive/MyDrive/{filenames}"
os.chdir(path2) #切換到要存取資料夾

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


###各種函式

In [ ]:
#合成照片
def denorm(img_tensors):
    return img_tensors * stats[1][0] + stats[0][0]

def show_images(images, nmax=64):
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.set_xticks([])
    ax.set_yticks([])

    #make_grid用於合成圖像
    ax.imshow(make_grid(denorm(images.detach()[:nmax]), nrow=8).permute(1, 2, 0))


def show_batch(dl, nmax=64):
    for images, _ in dl:
        show_images(images, nmax)
        break

#用GPU

def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device

    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl:
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

###下載資料庫&載入資料庫

In [ ]:
DATA_DIR = '/content/gdrive/MyDrive/anime_dataset/A'

train_ds = ImageFolder(DATA_DIR, transform=T.Compose([
    T.Resize(image_size),
    T.CenterCrop(image_size),
    T.ToTensor(),
    T.Normalize(*stats)]))

train_dl = DataLoader(train_ds, 128, shuffle=True, num_workers=3, pin_memory=True)

# for real_images, _ in tqdm(train_dl):
#     real_images = real_images.cpu()
#     print(real_images.shape)
#     for IM in real_images:
#       show_images(IM)
#     break

In [ ]:
# od.download(dataset_url)

In [ ]:
# from torchvision import transforms

# DATA_DIR = '/content/gdrive/MyDrive/anime_dataset/A'

# train_ds = ImageFolder(DATA_DIR, transform=T.Compose([
#     T.Resize(image_size),
#     T.CenterCrop(image_size),
#     T.ToTensor(),
#     T.Normalize(*stats)]))

# image, _ = train_ds[100]

# print(image.shape)

# image.permute(1, 2, 0)
# show_images(image)


# train_dl = DataLoader(train_ds, 1, shuffle=True, num_workers=3, pin_memory=True)

# new_size = (32, 32)
# transform = transforms.Compose([
#     transforms.Resize(new_size)
# ])

# for real_images, _ in tqdm(train_dl):
#     real_images = real_images.cpu()
#     print(real_images.shape)
#     image_resized = transform(real_images)
#     print(image_resized.shape)

#     for IM in image_resized:
#       show_images(IM)
#     break

#     for IM in real_images:
#       show_images(IM)
#     break
# # image2, _ = train_dl
# # print(image2.shape)

# # # show_batch(train_dl)#顯示資料庫64張照片

###判別器神經網路

In [ ]:
discriminator = nn.Sequential(
    # in: 3 x 64 x 64

    nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(64),
    nn.LeakyReLU(0.2, inplace=True),
    # out: 64 x 32 x 32

    nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(128),
    nn.LeakyReLU(0.2, inplace=True),
    # out: 128 x 16 x 16

    nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(256),
    nn.LeakyReLU(0.2, inplace=True),
    # out: 256 x 8 x 8

    nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(512),
    nn.LeakyReLU(0.2, inplace=True),
    # out: 512 x 4 x 4

    nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=0, bias=False),
    # out: 1 x 1 x 1

    nn.Flatten(),
    nn.Sigmoid())

###顯卡用
discriminator = discriminator.to('cuda')

###生成器神經網路

In [ ]:
generator  = nn.Sequential(
    # in: latent_size x 1 x 1

    nn.ConvTranspose2d(latent_size, 512, kernel_size=4, stride=1, padding=0, bias=False),
    nn.BatchNorm2d(512),
    nn.ReLU(True),
    # out: 512 x 4 x 4

    nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(256),
    nn.ReLU(True),
    # out: 256 x 8 x 8

    nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(128),
    nn.ReLU(True),
    # out: 128 x 16 x 16

    nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(64),
    nn.ReLU(True),
    # out: 64 x 32 x 32

    nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1, bias=False),
    nn.Tanh()
    # out: 3 x 64 x 64
)

###顯卡用
generator  = generator .to('cuda')

###顯卡裝置

In [ ]:
#顯卡是否能用
if torch.cuda.is_available():
    # 可以使用 CUDA
    print('GPU OK')
else:
    # 無法使用 CUDA
    print('haha, please use CPU')

GPU OK


In [ ]:
device = get_default_device()
train_dl = DeviceDataLoader(train_dl, device)
generator = to_device(generator, device)

In [ ]:
# for real_images, _ in tqdm(train_dl):
#     real_images = real_images.cpu()
#     print(real_images.shape)
#     for IM in real_images:
#       show_images(IM)
#     break
# # for real_images in train_dl[0]:
# #     real_images = real_images.cpu()
# #     show_images(real_images)

###算圖

In [ ]:
#定義判別器和生成器的判斷標準和誤差反向傳遞
generator = to_device(generator, device)

def train_discriminator(real_images, opt_d):
    # Clear discriminator gradients
    opt_d.zero_grad()

    # Pass real images through discriminator
    real_preds = discriminator(real_images)
    real_targets = torch.ones(real_images.size(0), 1, device=device)
    real_loss = F.MSELoss(real_preds, real_targets)
    real_score = torch.mean(real_preds).item()

    # Generate fake images
    latent = torch.randn(batch_size, latent_size, 1, 1, device=device)
    # print(latent.shape)
    fake_images = generator(latent)
    # print(fake_images.shape)

    # Pass fake images through discriminator
    fake_targets = torch.zeros(fake_images.size(0), 1, device=device)
    fake_preds = discriminator(fake_images)
    fake_loss = F.MSELoss(fake_preds, fake_targets)
    fake_score = torch.mean(fake_preds).item()

    # Update discriminator weights
    loss = real_loss + fake_loss
    loss.backward()
    opt_d.step()
    return loss.item(), real_score, fake_score

def train_generator(opt_g):
    # Clear generator gradients
    opt_g.zero_grad()

    # Generate fake images
    latent = torch.randn(batch_size, latent_size, 1, 1, device=device)
    fake_images = generator(latent)

    # Try to fool the discriminator
    preds = discriminator(fake_images)
    targets = torch.ones(batch_size, 1, device=device)
    loss = F.MSELoss(preds, targets)

    # Update generator weights
    loss.backward()
    opt_g.step()

    return loss.item()

#儲存算圖結果
def save_samples(index, latent_tensors, epoch, show=True):
    a = 0
    m =  f'epoch:{epoch}/'
    if not os.path.isdir(sample_dir+ '/' +m):
      os.mkdir(sample_dir+ '/' +m)
    fake_images = generator(latent_tensors)
    fake_fname = m +'generated-images-{0:0=4d}'.format(index)
    for a in range(10):
      save_image(denorm(fake_images[a]), os.path.join(sample_dir, fake_fname + str(a)+ '.png'))
      print('Saving', fake_fname)
    if show:
        fig, ax = plt.subplots(figsize=(8, 8))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(fake_images.cpu().detach(), nrow=8).permute(1, 2, 0))

fixed_latent = torch.randn(512, latent_size, 1, 1, device=device)
#算圖主程序
def fit(epochs, lr, start_idx=1):
    torch.cuda.empty_cache()

    # Losses & scores
    losses_g = []
    losses_d = []
    real_scores = []
    fake_scores = []

    # Create optimizers
    opt_d = torch.optim.ASGD(discriminator.parameters(), lr=lr, lambd=0.0005)
    opt_g = torch.optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))

    for epoch in range(epochs):
        for real_images, _ in tqdm(train_dl):
            # Train discriminator
            real_images = real_images.to('cuda')
            loss_d, real_score, fake_score = train_discriminator(real_images, opt_d)
            # Train generator
            loss_g = train_generator(opt_g)

        # Record losses & scores
        losses_g.append(loss_g)
        losses_d.append(loss_d)
        real_scores.append(real_score)
        fake_scores.append(fake_score)

        # Log losses & scores (last batch)
        print("Epoch [{}/{}], loss_g: {:.4f}, loss_d: {:.4f}, real_score: {:.4f}, fake_score: {:.4f}".format(
            epoch+1, epochs, loss_g, loss_d, real_score, fake_score))

        # Save generated images
        save_samples(epoch+start_idx, fixed_latent, epoch, show=True)

    return losses_g, losses_d, real_scores, fake_scores

losses_g, losses_d, real_scores, fake_scores = fit(epochs, lr)

# Save the model checkpoints
torch.save(generator.state_dict(), 'G.pth')
torch.save(discriminator.state_dict(), 'D.pth')

  0%|          | 0/111 [00:00<?, ?it/s]

Epoch [1/900], loss_g: 3.5339, loss_d: 0.9466, real_score: 0.5612, fake_score: 0.1639
Saving epoch:0/generated-images-0001
Saving epoch:0/generated-images-0001
Saving epoch:0/generated-images-0001
Saving epoch:0/generated-images-0001
Saving epoch:0/generated-images-0001
Saving epoch:0/generated-images-0001
Saving epoch:0/generated-images-0001
Saving epoch:0/generated-images-0001
Saving epoch:0/generated-images-0001
Saving epoch:0/generated-images-0001


  0%|          | 0/111 [00:00<?, ?it/s]

Epoch [2/900], loss_g: 3.5846, loss_d: 0.8065, real_score: 0.5761, fake_score: 0.0885
Saving epoch:1/generated-images-0002
Saving epoch:1/generated-images-0002
Saving epoch:1/generated-images-0002
Saving epoch:1/generated-images-0002
Saving epoch:1/generated-images-0002
Saving epoch:1/generated-images-0002
Saving epoch:1/generated-images-0002
Saving epoch:1/generated-images-0002
Saving epoch:1/generated-images-0002
Saving epoch:1/generated-images-0002


  0%|          | 0/111 [00:00<?, ?it/s]

...

  0%|          | 0/111 [00:00<?, ?it/s]

Epoch [245/900], loss_g: 5.8079, loss_d: 0.0247, real_score: 0.9897, fake_score: 0.0116
Saving epoch:244/generated-images-0245
Saving epoch:244/generated-images-0245
Saving epoch:244/generated-images-0245
Saving epoch:244/generated-images-0245
Saving epoch:244/generated-images-0245
Saving epoch:244/generated-images-0245
Saving epoch:244/generated-images-0245
Saving epoch:244/generated-images-0245
Saving epoch:244/generated-images-0245
Saving epoch:244/generated-images-0245


  0%|          | 0/111 [00:00<?, ?it/s]

Epoch [246/900], loss_g: 6.0422, loss_d: 0.0281, real_score: 0.9785, fake_score: 0.0059
Saving epoch:245/generated-images-0246
Saving epoch:245/generated-images-0246
Saving epoch:245/generated-images-0246
Saving epoch:245/generated-images-0246
Saving epoch:245/generated-images-0246
Saving epoch:245/generated-images-0246
Saving epoch:245/generated-images-0246
Saving epoch:245/generated-images-0246
Saving epoch:245/generated-images-0246
Saving epoch:245/generated-images-0246


  0%|          | 0/111 [00:00<?, ?it/s]

###顯示最終算圖圖片

In [ ]:
Image(f'./generated/generated-images-00{str(epochs)}.png')